In [1]:
import cv2
import numpy as np

# خواندن ویدیو
cap = cv2.VideoCapture('1232.mp4')

# بررسی اینکه ویدیو به درستی باز شده باشد
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

def region_of_interest(img):
    height = img.shape[0]
    polygons = np.array([[
        (100, height),
        (img.shape[1] - 100, height),
        (img.shape[1] // 2, height // 2)
    ]])
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, polygons, 255)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines):
    """ بهبود رسم خطوط با میانگین‌گیری و تثبیت خطوط جاده """
    left_lines = []
    right_lines = []
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                if x2 - x1 == 0:  # جلوگیری از تقسیم بر صفر
                    continue
                # شیب خط را محاسبه کنید
                slope = (y2 - y1) / (x2 - x1)
                if slope < -0.5:  # خطوط سمت چپ
                    left_lines.append((x1, y1, x2, y2))
                elif slope > 0.5:  # خطوط سمت راست
                    right_lines.append((x1, y1, x2, y2))

    # میانگین‌گیری و رسم خطوط سمت چپ و راست
    if left_lines:
        left_avg = np.mean(left_lines, axis=0).astype(int)
        cv2.line(img, (left_avg[0], left_avg[1]), (left_avg[2], left_avg[3]), (0, 255, 0), 10)
    if right_lines:
        right_avg = np.mean(right_lines, axis=0).astype(int)
        cv2.line(img, (right_avg[0], right_avg[1]), (right_avg[2], right_avg[3]), (0, 255, 0), 10)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame or end of video.")
        break

    # تغییر اندازه فریم
    scale_percent = 50
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized_frame = cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)

    # تبدیل به تصویر خاکستری
    gray = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)

    # استفاده از GaussianBlur برای حذف نویز
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # استفاده از Canny برای تشخیص لبه‌ها
    canny = cv2.Canny(blurred, 50, 150)

    # انتخاب ناحیه مثلثی جاده
    cropped_canny = region_of_interest(canny)

    # تشخیص خطوط با تبدیل هاف
    lines = cv2.HoughLinesP(cropped_canny, 1, np.pi / 180, threshold=50, minLineLength=40, maxLineGap=150)

    # رسم خطوط میانگین‌گیری شده
    draw_lines(resized_frame, lines)

    # نمایش فریم
    cv2.imshow('Lane Detection', resized_frame)

    # خروج با کلید 'q'
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# آزادسازی منابع
cap.release()
cv2.destroyAllWindows()
